In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras import backend as K
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

np.random.seed(2017)

train = pd.read_csv('fashion-mnist_train.csv', delimiter=',')
test  = pd.read_csv('test_data.csv', delimiter=',')


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7218080035960822973
]


Using TensorFlow backend.


FileNotFoundError: File b'fashion-mnist_train.csv' does not exist

In [ ]:
X_train = train.as_matrix()
X_train = np.reshape(X_train[:,1:785],(60000,28,28,1))
print(X_train.shape)

y_train = train.ix[:,0]
print(y_train.shape)

X_test = test.as_matrix()
X_test = np.reshape(X_test[:,1:785],(10000,28,28,1))
print(X_test.shape)


Y_train = np_utils.to_categorical(y_train, 10)


print('train labels:\t',Y_train.shape)
print(Y_train[0:10, :])

trainX_normal = (X_train/255.)
testX_normal = (X_test/255.)

trainX_normal = trainX_normal.astype('float32')
testX_normal = testX_normal.astype('float32')

print('train)\tmin: ', np.min(trainX_normal), '\tmax: ', np.max(trainX_normal))
print('test)\tmin: ', np.min(testX_normal), '\tmax: ', np.max(testX_normal))
print(trainX_normal.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(trainX_normal, Y_train,
          batch_size=128,
          epochs=100,
          verbose=1)

In [ ]:
prd1 = model.predict(testX_normal)
prd1_y = np.argmax(prd1, axis=1)
print(prd1_y[0:10])

In [ ]:
np.savetxt("CNN1.csv", prd1_y, delimiter=",") # yield 0.93 on kaggle
model.save('./CNN1.h5') # creates a HDF5 file 'CNN1.h5'